In [1]:
# General imports
import os
import sys
import pandas as pd
import scipy
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from tqdm import tqdm
sys.path.append('../')
# Other imports

tqdm.pandas()
import yaml
import json

# Figures imports
import matplotlib
from matplotlib.lines import Line2D   
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from statannot import add_stat_annotation
import matplotlib.font_manager as font_manager
import string


## YAML FILES CONFIG
yaml = yaml.load(open("config/config_files.yaml"), Loader=yaml.FullLoader)
base_dir = yaml['base_directory']
sys.exit('EXIT : Need to specify the base_directory in config file : "conf_files.yaml"') if base_dir == 'TO_CHANGE' else None
    


# Font settings
font_dirs = ['/home/weber/Fonts', ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

pvalues_cutoff = json.load(open('config/config_files.json'))['pvalues_cutoff']
pvalues_cutoff = [[e, str(e)] for e in pvalues_cutoff]

import matplotlib.font_manager as font_manager

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['font.weight'] = 'light'
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 18

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:38: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.


# StringDB + Clustering

Liste des fichiers précédemment générés : 

In [324]:
! ls -l /gstock/GeneIso/V2_with_MM_expression/Gene_lists_STRING

total 76
-rw-r--r-- 1 weber bio3d 34722 mars   7 14:56 HS-Miso-MM-Miso_HS_gene_names.txt
-rw-r--r-- 1 weber bio3d 20338 mars   7 14:56 HS-Miso-MM-Siso_HS_gene_names.txt
-rw-r--r-- 1 weber bio3d  7202 mars   7 14:56 HS-Siso-MM-Miso_HS_gene_names.txt
-rw-r--r-- 1 weber bio3d 11859 mars   7 14:56 HS-Siso-MM-Siso_HS_gene_names.txt


## StringDB

Ligne commande pour passer d'une liste de gènes à intéractome pour chacun des fichiers
Seuil 900 utilisé
```
> setgscoperr
> gscope puts StringInteractome FichierContenantListeGene RepertoireSortie Organisme Seuil
> Example : gscope puts StringInteractome /gstock/GeneIso/V2/Gene_lists_STRING/test.txt /gstock/GeneIso/V2/STRING_RESULTS/ 9606 900
```

## MCL

 Traitement des QueriesOnly pour ne pas ramener des gènes avec d'autres statuts que ceux dans la liste
 
 ```
/biolo/mcl/bin/mcl <(cat QueryOnly.string | cut -f 2,4) --abc -I 2.0 -o "$OUTPUT"
```

# HS Miso - MM Siso

## Queries only after MM expression


In [327]:
# Retrieve MCL results & transform into dict : {cluster : genelist}

clustering_dict = pd.read_csv('/gstock/GeneIso/V2_with_MM_expression/STRING_RESULTS/HS-Miso-MM-Siso_HS_gene_names/String_9606_202203071501/QueryOnly_20.abc', names=['Cluster'])
clustering_dict.index = clustering_dict.index  + 1
clustering_dict = clustering_dict.to_dict()['Cluster']
clustering_dict = {k:sorted(v.split('\t')) for k,v in clustering_dict.items()}


In [326]:
for cluster, cluster_content in clustering_dict.items():
    if cluster <= 15:
        print('>Cluster {} - {} gènes'.format(cluster, len(cluster_content)))
        print(" ".join(cluster_content))
    
    

>Cluster 1 - 238 gènes
ACAA1 ADRM1 AGTR1 AHCTF1 AMACR ANAPC4 AP2A2 ARIH2 ASB14 ASB6 ASB7 ATRIP BABAM1 BACH1 BAP1 BARD1 BRCA1 BUB1 C17orf70 CACYBP CCNB2 CDC16 CDC20 CDCA3 CENPP CENPT CHD1L CHMP1A CHMP5 CHMP7 CHTF8 CKAP2 CLHC1 COMMD4 COMMD5 COMMD9 COPS3 COPS4 COPS5 COPS8 CROT DCAF10 DCAF5 DCAF6 DCDC2 DERL1 DTL DTX1 DYRK2 EGLN3 EIF4E3 ENGASE EPM2A ERLEC1 ERLIN2 FAM64A FAN1 FANCM FBXL16 FBXL2 FBXL20 FBXL22 FBXL4 FBXL7 FBXL8 FBXO18 FBXO31 FBXO32 FBXO4 FBXO5 FBXW8 FBXW9 GADD45G GAK GEN1 GMDS GSK3A GSTK1 GYS1 HECTD3 HECW1 HELQ HERC6 HLTF HMGCL HSD17B4 HUS1 IDE IREB2 ITGB3BP JAG2 KCTD7 KDM1B KIAA0101 KIAA0319 KLHL21 KLHL3 KNTC1 LDLR LGR6 LIMD1 LONRF1 LRIG1 LRRC41 LTA LTBR MAD1L1 MAD2L1BP MAP2K4 MAP3K14 MAP3K8 MCM3 MELK MIB2 MLH1 MSH2 MSH6 MUS81 MVB12A MVB12B MYSM1 NECAP1 NECAP2 NFRKB NHEJ1 OAZ3 ODC1 ORC1 ORC5 OTUD7A PARP2 PCNA PEX10 PEX11A PEX13 PEX16 PEX6 PHYH PKMYT1 PLAA PLK2 PMS1 POLA1 POLB POLD1 POLD2 POLE2 POLE3 POLH POLL POLN PRKDC PSMA2 PSMA4 PSMA6 PSMB2 PSMB6 PSMB8 PSMC1 PSMC2 PSMC3IP 

# HS Siso - MM Miso

In [329]:
clustering_dict = pd.read_csv('/gstock/GeneIso/V2_with_MM_expression/STRING_RESULTS/HS-Siso-MM-Miso_HS_gene_names/String_9606_202203071503/QueryOnly_20.abc', names=['Cluster'])
clustering_dict.index = clustering_dict.index  + 1
clustering_dict = clustering_dict.to_dict()['Cluster']
clustering_dict = {k:sorted(v.split('\t')) for k,v in clustering_dict.items()}
all_genes_clustering = [sub_e for k in clustering_dict.keys() for sub_e in clustering_dict[k] ]


In [330]:
for cluster, cluster_content in clustering_dict.items():
    if cluster <= 15:
        print('>Cluster {} - {} gènes'.format(cluster, len(cluster_content)))
        print(" ".join(cluster_content))
    
    

>Cluster 1 - 41 gènes
AQR CASC3 CDK9 CWC15 CWC22 CWC25 DHX15 DHX9 DNAJC8 EDC4 ELL2 ELL3 GTF2E2 HNRNPH2 INTS1 INTS10 INTS8 IWS1 LSM10 LSM2 LSM3 METTL14 NUCKS1 NUDT21 PHAX POLR2C POLR2G POLR2I POLR2J PRPF18 PRPF19 PRPF3 PRPF40A PRPF6 RPAP2 RPRD1B SF3A2 SLU7 SNRNP48 SNRPA XPA
>Cluster 2 - 31 gènes
ACVR2B BMPR1A CDC26 CDC34 COMMD1 COMMD3 CUL5 DCAF15 DDA1 FBXL15 FBXL3 FBXO30 FBXO6 GLMN HERC2 KDM4A KLHL22 KLHL25 RBX1 RNF144B RNF168 RNF20 RNF25 SH3RF1 SKI SMURF2 TRIM21 UBE2B UBE2M UBE2Q1 ZER1
>Cluster 3 - 24 gènes
DDX27 EIF2S2 EIF2S3 GRWD1 MRTO4 NIFK NOL11 NOL7 NOL9 NOP56 NOP58 NSUN6 RBM34 RPF2 RPL10A RPL3L RPS25 RPS28 RRP36 SURF6 TRUB1 TSR2 UTP14C UTP6
>Cluster 4 - 21 gènes
ADCY9 ADRB3 ADRBK2 C5AR1 CHRM1 CXCL10 FZD5 GNA14 GNAZ GNG12 GNG7 GPR37 GRK5 NTSR2 OXTR P2RY12 PDCL PSAP PTGER4 RGS2 WNT11
>Cluster 5 - 18 gènes
ARF5 COG1 COPG1 GCC1 GOLGA1 GOLGA5 KIF1C LMAN1 MAN1A1 NAPG PEF1 RGP1 RINT1 SEC22B SEC31B TMED10 TMED7 TMED9
>Cluster 6 - 17 gènes
CRKL FGF18 FGF9 JAK2 MAP2K2 NRAS PAPSS1 PIM1 PLA2

# GPROFILER / ENRICHMENT ANALYSIS

In [310]:
mm_genes = pd.read_csv('/gstock/GeneIso/V2_with_MM_expression/misog_sisog_mm_expr.tsv.gz', compression='gzip', sep='\t')
mm_genes_clean = mm_genes.loc[mm_genes['Miso_siso_mm'] == mm_genes['Miso_siso_mm_expr']]
mm_genes_clean

,GeneID,ensembl_gene_id,Miso_siso_mm,Miso_siso_mm_expr
0,ENSG00000000003,ENSMUSG00000067377,Siso,Siso
1,ENSG00000000005,ENSMUSG00000031250,Siso,Siso
2,ENSG00000000419,ENSMUSG00000078919,Miso,Miso
3,ENSG00000000457,ENSMUSG00000026584,Miso,Miso
4,ENSG00000000460,ENSMUSG00000041406,Miso,Miso
...,...,...,...,...
11688,ENSG00000284194,ENSMUSG00000091780,Miso,Miso
11689,ENSG00000284308,ENSMUSG00000030030,Miso,Miso
11690,ENSG00000284753,ENSMUSG00000115219,Miso,Miso
11691,ENSG00000284770,ENSMUSG00000039233,Siso,Siso


In [332]:
hs_genes = pd.read_parquet("/gstock/GeneIso/V2/Genes.parquet")
hs_genes

,GeneID,transcript_count,Miso_siso,Gene name,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene_length
0,ENSG00000000003,3,Miso,TSPAN6,X,100627108,100639991,12883
1,ENSG00000000005,1,Siso,TNMD,X,100584936,100599885,14949
2,ENSG00000000419,3,Miso,DPM1,20,50934867,50959140,24273
3,ENSG00000000457,3,Miso,SCYL3,1,169849631,169894267,44636
4,ENSG00000000460,4,Miso,C1orf112,1,169662007,169854080,192073
...,...,...,...,...,...,...,...,...
13496,ENSG00000286143,1,Siso,None,2,219491867,219498244,6377
13497,ENSG00000286185,1,Siso,None,1,149390623,149556361,165738
13498,ENSG00000286190,1,Siso,None,17,5499427,5501006,1579
13499,ENSG00000287542,1,Siso,None,4,88523810,88708450,184640


### Compute MISOG/SISOG ratio for HS & MM in order to compare by GO

In [333]:
misog_hs = set(hs_genes.loc[hs_genes['Miso_siso'] == 'Miso', 'GeneID'].values.tolist())
sisog_hs = set(hs_genes.loc[hs_genes['Miso_siso'] == 'Siso', 'GeneID'].values.tolist())
print(len(misog_hs))
print(len(sisog_hs))
print(len(misog_hs) / len(sisog_hs)) 

9826
3675
2.6737414965986392


In [334]:
misog_mm = set(mm_genes_clean.loc[mm_genes_clean['Miso_siso_mm'] == 'Miso', 'ensembl_gene_id'].values.tolist())
sisog_mm = set(mm_genes_clean.loc[mm_genes_clean['Miso_siso_mm'] == 'Siso', 'ensembl_gene_id'].values.tolist())
print(len(misog_mm))
print(len(sisog_mm))
print(len(misog_mm) / len(sisog_mm)) 

6601
4485
1.471794871794872


In [335]:

def stats_misog_sisog(r, specie):
    """
    Compute ratio & normalize according specie MISOG/SISOG ratio
    """
    df = pd.Series()
    ensg = set(r['converted'].values.tolist())
    
    if specie == 'hs':
        df['HS_nb_ensg_total_in_term'] = len(ensg)
        df['HS_nb_ensg_misog'] = len(ensg.intersection(misog_hs))
        df['HS_nb_ensg_sisog'] = len(ensg.intersection(sisog_hs))
        df['HS_ratio_misog_sisog'] = (df['HS_nb_ensg_misog'] / df['HS_nb_ensg_sisog']) / (len(misog_hs) / len(sisog_hs))
    elif specie == 'mm':
        df['MM_nb_ensg_total_in_term'] = len(ensg)
        df['MM_nb_ensg_misog'] = len(ensg.intersection(misog_mm))
        df['MM_nb_ensg_sisog'] = len(ensg.intersection(sisog_mm))
        df['MM_ratio_misog_sisog'] = (df['MM_nb_ensg_misog'] / df['MM_nb_ensg_sisog']) / (len(misog_mm) / len(sisog_mm))        
#     print(df)
    return df


def profiler_enrichment(gene_list, sources=['GO:MF', 'GO:BP','GO:CC', 'REAC', 'KEGG', 'WP', 'CORUM', 'HP'], correction_method='fdr'):
    """
    Function to retrieve enrichment based on gene list and defined sources & correction (default : all sources, BH FDR correction)
    """
    gp = GProfiler(return_dataframe=True)
    
    # Enrichment profiling according list
    df = gp.profile(organism='hsapiens',
               sources=sources,
                query=gene_list,
              combined=False, no_evidences=False)
    
    # Use gProfiler convert tool to retrieve all associated genes for each term
    df_go_hs = gp.convert(organism='hsapiens', query=df['native'].values.tolist())
    # groupby term & apply for each term the stats_misog_sisog fct to compute ratio 
    df_go_hs = df_go_hs.groupby('incoming').progress_apply(lambda r : stats_misog_sisog(r, 'hs'))
    
    # Same for MM
    df_go_mm = gp.convert(organism='mmusculus', query=df['native'].values.tolist())
    df_go_mm = df_go_mm.groupby('incoming').progress_apply(lambda r : stats_misog_sisog(r, 'mm'))    
    
    # Merge dataframes
    df = pd.merge(
            pd.merge(
                df, df_go_hs.reset_index(), left_on='native', right_on='incoming'
            ),
            df_go_mm,
            on='incoming'
    )
    
    return df




## ANALYSIS FOR GENES IDENTIFIED IN CLUSTERS WITH MCL BASED ON STRING INTERACTOME
### HS MISOG > MM SISOG


In [ ]:
# Retrieve genes
clustering_dict = pd.read_csv('/gstock/GeneIso/V2/HS-Miso-MM-Siso_HS_gene_names/String_9606_202202251921/QueryOnly_20.abc', names=['Cluster'])
clustering_dict.index = clustering_dict.index  + 1
clustering_dict = clustering_dict.to_dict()['Cluster']
clustering_dict = {k:v.split('\t') for k,v in clustering_dict.items()}


writer = pd.ExcelWriter('/gstock/GeneIso/V2/HS-Miso-MM-Siso_HS_gene_names/String_9606_202202251921/GO_enrichment_v2.xlsx', engine='xlsxwriter')

complete_df_enrichment = list()
index_list = list()
sources = ['GO:MF', 'GO:BP','GO:CC', 'REAC', 'KEGG', 'WP', 'CORUM', 'HP']

# For each cluster, perform term enrichment & retrieve ratio in order to identify atypic ratios
for k,v in clustering_dict.items():
    if len(v) >= 20:
        print(k, len(v))
        tmp_df = profiler_enrichment(clustering_dict[k], sources=sources)
        tmp_df.to_excel(writer,'Cluster {}'.format(k), index=False)
writer.save()        


## ANALYSIS FOR GENES IDENTIFIED IN CLUSTERS WITH MCL BASED ON STRING INTERACTOME
### HS SISOG > MM MISOG


In [336]:
# Retrieve genes
clustering_dict = pd.read_csv('/gstock/GeneIso/V2/HS-Siso-MM-Miso_HS_gene_names/String_9606_202202251921/QueryOnly_20.abc', names=['Cluster'])
clustering_dict.index = clustering_dict.index  + 1
clustering_dict = clustering_dict.to_dict()['Cluster']
clustering_dict = {k:v.split('\t') for k,v in clustering_dict.items()}


writer = pd.ExcelWriter('/gstock/GeneIso/V2/HS-Siso-MM-Miso_HS_gene_names/String_9606_202202251921/GO_enrichment_v2.xlsx', engine='xlsxwriter')

complete_df_enrichment = list()
index_list = list()
sources = ['GO:MF', 'GO:BP','GO:CC', 'REAC', 'KEGG', 'WP', 'CORUM', 'HP']
for k,v in clustering_dict.items():
    if len(v) >= 10:
        print(k, len(v))
        tmp_df = profiler_enrichment(clustering_dict[k], sources=sources)
        tmp_df.to_excel(writer,'Cluster {}'.format(k), index=False)
writer.save()        

1 42


  0%|                                                                                                             | 0/148 [00:00<?, ?it/s]/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 148/148 [00:00<00:00, 220.35it/s]


2 33


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 164/164 [00:00<00:00, 239.55it/s]


3 32


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 147.81it/s]


15 10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 253.83it/s]


16 10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 265.06it/s]
